# SimPy Tutorial Notebook: Building Discrete Event Simulations (DES) Incrementally

This file provides a structured, multi-part tutorial for SimPy, a process-based
discrete-event simulation framework for Python. We will progress from basic
concepts (Environment, Processes) to complex features (Resources, Priority,
Containers, and Data Collection).

Key Modules Used:
- simpy: The core DES framework.
- random: For generating stochastic (random) event times.
- statistics: For basic data analysis.

In [ ]:
import simpy
import random
import statistics

# --- Global Configuration and Setup ---
# Setting a seed ensures that the "random" events are the same every time you run the simulation,
# which is CRITICAL for debugging and comparing model changes.
RANDOM_SEED = 42
SIM_DURATION = 1000  # Total simulation time in abstract units (e.g., minutes)

print("--- SimPy Tutorial Notebook ---")
print(f"Setting random seed to {RANDOM_SEED} for reproducibility.")
random.seed(RANDOM_SEED)

## PART 1: CORE CONCEPTS (Environment and Processes)
Example: A Simple Car Process

Concepts Covered:
1. simpy.Environment: The scheduler and clock.
2. Generator Functions (Processes): The active entities in the simulation.
3. env.timeout(): Pausing a process for a specified amount of simulation time.

In [ ]:
def car(env, name, parking_duration, trip_duration):
    """
    A process modeling a car that drives, parks, and drives again.
    
    The 'env' (environment) is always the first argument in a SimPy process.
    """
    # Loop indefinitely to show the process repeating
    while True:
        # 1. Driving Phase
        print(f"[{env.now:.2f}] Car {name}: Starting trip. Duration: {trip_duration} min.")
        
        # 'yield env.timeout()' pauses this process and tells SimPy to fast-forward
        # the clock until 'trip_duration' has elapsed.
        yield env.timeout(trip_duration)

        # 2. Parking Phase
        print(f"[{env.now:.2f}] Car {name}: Arrived, starting to park. Duration: {parking_duration} min.")
        yield env.timeout(parking_duration)
        
        # 3. Ready to drive again
        print(f"[{env.now:.2f}] Car {name}: Parking finished. Getting ready to drive.")

def run_part1():
    """Sets up and runs the simple process simulation."""
    print("\n\n=== PART 1: Environment and Processes (The Simple Timer) ===")
    
    # 1. Create the Environment
    env = simpy.Environment()
    
    # 2. Start the Processes
    # We define three cars with different driving/parking habits.
    env.process(car(env, 'BMW', parking_duration=5, trip_duration=15))
    env.process(car(env, 'Audi', parking_duration=8, trip_duration=20))
    env.process(car(env, 'Tesla', parking_duration=3, trip_duration=10))

    # 3. Run the Simulation
    print(f"\nRunning simulation until t={50}...")
    env.run(until=50)

In [ ]:
run_part1()

## PART 2: BASIC RESOURCES (Limited Capacity and Queuing)
Example: The Single Server Checkout Line

Concepts Covered:
 1. simpy.Resource: Models a limited-capacity service point (the Server).
 2. resource.request(): Event for acquiring a resource unit (joining the queue).
 3. resource.release(): Event for giving the resource unit back (finishing service).
 4. Context Manager (with): The cleanest way to handle request/release.
 5. Customer Generation: Using a process to feed entities into the system.

In [ ]:
# Constants for Part 2
NUM_SERVERS = 2         # Number of available checkout counters
T_INTER = 6             # Mean time between customer arrivals (minutes)
T_SERVICE = 3           # Mean service time at the counter (minutes)


In [ ]:
def customer(env, name, server):
    """The customer process that requests and releases the server resource."""
    arrival_time = env.now
    print(f"[{env.now:.2f}] Customer {name}: Arrived. Requesting a server.")

    # Request the resource using a context manager (`with` statement).
    # This automatically handles the release even if an error occurs.
    with server.request() as req:
        # Pauses until a server is available (yields the request event)
        yield req
        
        # Server is now acquired!
        wait_time = env.now - arrival_time
        print(f"[{env.now:.2f}] Customer {name}: Acquired server. Waited {wait_time:.2f} min. Starting service.")

        # Simulate the service time
        service_time = max(1, random.expovariate(1.0 / T_SERVICE))
        yield env.timeout(service_time)
        sojourn_time = env.now - arrival_time
        print(f"[{env.now:.2f}] Customer {name}: Finished service in {service_time:.2f} min. Total time in system: {sojourn_time:.2f} min.")
        
    # The 'with' block automatically called 'server.release(req)' here.

def customer_generator(env, server):
    """Process to continuously generate customers at random intervals."""
    cust_id = 0
    while True:
        # Generate the next customer (Process)
        cust_id += 1
        env.process(customer(env, cust_id, server))
        
        # Schedule the next customer arrival
        t_next_arrival = random.expovariate(1.0 / T_INTER)
        yield env.timeout(t_next_arrival) # Pauses the generator process

def run_part2():
    """Sets up and runs the basic resource simulation."""
    print("\n\n=== PART 2: Basic Resources (The Single Server Checkout Line) ===")
    
    # 1. Setup
    env = simpy.Environment()
    
    # 2. Define the Resource (2 servers available)
    server = simpy.Resource(env, capacity=NUM_SERVERS)
    print(f"Setup: {NUM_SERVERS} servers available.")
    
    # 3. Start the generator process
    env.process(customer_generator(env, server))

    # 4. Run the simulation
    print(f"\nRunning simulation until t={SIM_DURATION}...")
    env.run(until=SIM_DURATION)

In [ ]:
run_part2()

## PART 3: ADVANCED RESOURCES (Priority and Preemption)
 Example: Emergency Room Triage

 Concepts Covered:
 1. simpy.PriorityResource: Queues based on priority, not just arrival time.
 2. Preemption: Kicking a low-priority process off the resource.
 3. Interrupts: How the kicked process handles being interrupted.

In [ ]:
# Constants for Part 3
NUM_DOCTORS = 1
EMERGENCY_PROB = 0.2    # Probability of a critical patient
CRITICAL_PRIORITY = 0   # Lower number = Higher priority
NORMAL_PRIORITY = 10

In [ ]:
def patient(env, name, doctor_resource):
    """Patient process using a PriorityResource."""
    is_critical = random.random() < EMERGENCY_PROB
    priority = CRITICAL_PRIORITY if is_critical else NORMAL_PRIORITY
    
    # Preempt is True: A critical patient can interrupt a normal one.
    preempt_flag = True if is_critical else False
    
    print(f"[{env.now:.2f}] Patient {name} (Prio: {priority}): Arrived. Critical: {is_critical}. Seeking doctor.")

    try:
        # Request the PriorityResource, providing the priority key and preemption flag.
        with doctor_resource.request(priority=priority, preempt=preempt_flag) as req:
            
            # The yield will pause here until the request is fulfilled.
            yield req 
            
            # Doctor acquired!
            service_time = random.uniform(5, 15) # Longer service for simplicity
            print(f"[{env.now:.2f}] Patient {name}: Acquired doctor. Start consultation ({service_time:.2f} min).")
            
            # Consultation (Timeout Event)
            yield env.timeout(service_time) 

            print(f"[{env.now:.2f}] Patient {name}: Finished and released the doctor.")
            
    except simpy.Interrupt as i:
        # This block executes ONLY if the process was preempted (kicked out).
        cause = i.cause
        
        # Get the time used before preemption
        time_used = env.now - cause.usage_since
        
        print(f"[{env.now:.2f}] *** INTERRUPTED *** Patient {name} was preempted by {cause.by.name}.")
        print(f"[{env.now:.2f}] Patient {name}: Used {time_used:.2f} min. Re-queuing now.")
        
        # In a real model, we would save the remaining service time and re-request
        # the resource. For this example, we'll just let the patient leave.

def patient_generator(env, resource):
    """Generates patients continuously."""
    p_id = 0
    while True:
        p_id += 1
        env.process(patient(env, p_id, resource))
        
        # Wait for next arrival (random interval)
        yield env.timeout(random.uniform(3, 8))

def run_part3():
    """Sets up and runs the advanced resource simulation."""
    print("\n\n=== PART 3: Advanced Resources (ER Triage - Preemption) ===")
    
    # 1. Setup Environment and PriorityResource
    env = simpy.Environment()
    # Note: Use PreemptiveResource to allow interruption of currently-using processes.
    doctor_resource = simpy.PreemptiveResource(env, capacity=NUM_DOCTORS)
    
    print(f"Setup: {NUM_DOCTORS} Doctor (Preemptive) available. Low Prio = Normal ({NORMAL_PRIORITY}), High Prio = Critical ({CRITICAL_PRIORITY}).")
    
    # 2. Start the generator process
    env.process(patient_generator(env, doctor_resource))

    # 3. Run the simulation
    print(f"\nRunning simulation until t={SIM_DURATION/2} (500 min)...")
    env.run(until=SIM_DURATION/2)

In [ ]:
run_part3()

## PART 4: INVENTORY AND LOGISTICS (Containers)
Example: The Refinery Fuel Tank

Concepts Covered:
1. simpy.Container: Models the consumption (get) and production (put) of bulk goods.
2. container.get() / container.put(): Events that wait for capacity/content.
3. Multiple Producer/Consumer processes.    

In [ ]:
# Constants for Part 4
TANK_CAPACITY = 1000  # Max fuel storage
TANK_INIT = 500       # Initial fuel level
REFILL_RATE = 100     # Amount produced per production cycle
CONSUME_RATE = 50     # Amount consumed per consumption cycle

In [ ]:
def consumer(env, tank):
    """Process representing a refinery consuming fuel."""
    while True:
        print(f"[{env.now:.2f}] Consumer: Tank has {tank.level} units. Requesting {CONSUME_RATE} units...")

        # Request event: Wait until the tank has at least CONSUME_RATE units.
        yield tank.get(CONSUME_RATE)
        
        print(f"[{env.now:.2f}] Consumer: Successfully consumed {CONSUME_RATE} units. Tank level: {tank.level}")
        
        # Wait a random time before consuming again
        yield env.timeout(random.uniform(5, 10))

def producer(env, tank):
    """Process representing the refinery producing fuel."""
    while True:
        # Wait a random time before producing
        yield env.timeout(random.uniform(10, 20))
        
        print(f"[{env.now:.2f}] Producer: Tank has {tank.level} units. Refilling {REFILL_RATE} units...")

        # Put event: Wait until there is space for REFILL_RATE units.
        yield tank.put(REFILL_RATE)
        
        print(f"[{env.now:.2f}] Producer: Successfully refilled {REFILL_RATE} units. Tank level: {tank.level}")


def run_part4():
    """Sets up and runs the container simulation."""
    print("\n\n=== PART 4: Inventory & Logistics (The Fuel Tank) ===")
    
    # 1. Setup Environment and Container
    env = simpy.Environment()
    # The Container is a resource that holds bulk material.
    fuel_tank = simpy.Container(env, capacity=TANK_CAPACITY, init=TANK_INIT)
    
    print(f"Setup: Tank Capacity={TANK_CAPACITY}, Initial Level={TANK_INIT}.")
    
    # 2. Start the producer and consumer processes
    env.process(consumer(env, fuel_tank))
    env.process(producer(env, fuel_tank))

    # 3. Run the simulation
    print(f"\nRunning simulation until t={SIM_DURATION/3} (333.33 min)...")
    env.run(until=SIM_DURATION/3)

In [ ]:
run_part4()

## PART 5: DATA COLLECTION AND ANALYSIS
Example: Analyzing the Checkout Line as a Single Server System (M/M/1)

Concepts Covered:
 1. Data Collection: Recording metrics inside processes.
 2. Simulation Results: Calculating and displaying key Queuing Theory statistics.
 3. M/M/1 Configuration: Setting resource capacity to 1 (one server).

In [ ]:
# Data structure to hold customer-level results (Wait Time, Sojourn Time)
results_log = [] 
# Data structure to hold system-level results (Queue Size, Utilization)
queue_size_log = [] 

def analyzed_customer(env, name, server):
    """
    The customer process, now recording wait time, service time, and sojourn time.
    """
    arrival_time = env.now
    
    with server.request() as req:
        # 1. Wait for the server (queuing time)
        yield req
        
        # 2. Calculate Wait Time
        wait = env.now - arrival_time
        
        # 3. Service
        service_start_time = env.now
        service_time = max(1, random.expovariate(1.0 / T_SERVICE))
        yield env.timeout(service_time)

        # 4. Calculate Sojourn Time (Total time in system: Wait + Service)
        sojourn = env.now - arrival_time
        
        # 5. Record all metrics
        results_log.append({
            'wait_time': wait,
            'service_time': service_time,
            'sojourn_time': sojourn,
            'arrival_time': arrival_time,
            'departure_time': env.now
        })

def analyzed_generator(env, server):
    """Process to generate customers for data analysis."""
    cust_id = 0
    # We stop the generator before the main simulation run to ensure all customers
    # are generated before analysis.
    max_customers = 200
    
    for _ in range(max_customers):
        cust_id += 1
        env.process(analyzed_customer(env, cust_id, server))
        
        t_next_arrival = random.expovariate(1.0 / T_INTER)
        yield env.timeout(t_next_arrival)

def monitor_resource(env, resource):
    """
    Periodically logs the state of the resource (queue size and usage).
    This process is essential for calculating Lq and L (average number of customers).
    """
    # Sample the resource state every 1.0 time unit
    while True:
        queue_size_log.append({
            'time': env.now,
            'queue': len(resource.queue),
            'users': len(resource.users)
        })
        yield env.timeout(1.0)


def run_part5():
    """
    Sets up and runs the data collection simulation as an M/M/1 system,
    then calculates and prints the key queuing statistics.
    """
    # --- M/M/1 Configuration ---
    # We set the capacity to 1 to simulate a single-server system.
    SINGLE_SERVER_CAPACITY = 1
    
    print("\n\n=== PART 5: M/M/1 Queuing System Analysis ===")
    print(f"Configuration: Single Server (M/M/1), Arrival Rate (1/λ): {T_INTER}, Service Rate (1/μ): {T_SERVICE}")
    
    # Reset data from previous runs if any
    results_log.clear()
    queue_size_log.clear()
    
    # 1. Setup
    env = simpy.Environment()
    # Explicitly set capacity=1 for the M/M/1 model
    server = simpy.Resource(env, capacity=SINGLE_SERVER_CAPACITY)
    
    # 2. Start the generator and the monitor
    env.process(analyzed_generator(env, server))
    env.process(monitor_resource(env, server)) # Tracks Lq and L
    
    # 3. Run the simulation
    print(f"\nRunning simulation until t={SIM_DURATION}...")
    env.run(until=SIM_DURATION)
    
    # 4. Analysis and Queuing Statistics Output
    print("\n--- Queuing Statistics (M/M/1 Simulation Results) ---")
    
    if results_log:
        wait_times = [r['wait_time'] for r in results_log]
        sojourn_times = [r['sojourn_time'] for r in results_log]
        
        print(f"Total customers served: {len(results_log)}")
        
        # Wq: Mean Waiting Time (in queue)
        mean_wait_time = statistics.mean(wait_times)
        print(f"1. Mean Waiting Time ($W_q$): {mean_wait_time:.2f} minutes")
        
        # W: Mean Sojourn Time (in system)
        mean_sojourn_time = statistics.mean(sojourn_times)
        print(f"2. Mean Sojourn Time ($W$): {mean_sojourn_time:.2f} minutes")
        
    else:
        print("No customer data collected.")
    
    if queue_size_log:
        # Lq: Average number of queued tasks
        # Note: We are using a simple average of periodic samples (t=1.0) 
        # as a stand-in for the true time-weighted average, which is more complex to calculate manually.
        queue_lengths = [d['queue'] for d in queue_size_log]
        avg_queue_size = statistics.mean(queue_lengths)
        print(f"3. Average Queue Size ($L_q$): {avg_queue_size:.2f} customers")
        
        # L: Average number of tasks in the system (Queue + In Service)
        system_lengths = [d['queue'] + d['users'] for d in queue_size_log]
        avg_system_size = statistics.mean(system_lengths)
        print(f"4. Average System Size ($L$): {avg_system_size:.2f} customers")

In [ ]:
run_part5()